In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

## 夏普比率

In [3]:
data = np.random.normal(size=100)

rf = 0.03

# 年化夏普比率
sharpe = (data.mean()-rf) / data.std(ddof=1) * np.sqrt(252)

print(sharpe)

2.5824610244740964


## 最大回撤

In [5]:
data = np.random.lognormal(0, 0.5, 100)

# 找到区间截至每日的最大复权单位净值
acc_max = np.maximum.accumulate(data)

# 每日当前回撤
dd = (data - acc_max) / acc_max

# 区间最大回撤结束位置
j = np.argmin(dd)

# 区间最大回撤开始位置
i = np.argmax(data[:j]) if j != 0 else 0

max_dd = (data[j] - data[i]) / data[i]

print(max_dd, dd.min())

-0.8865163316678305 -0.8865163316678305


## 年化卡玛比率 Calmar Ratio

In [22]:
# method1: 已知价格求解
r = data[-1] / data[0]  # 注意这里不是收益率，收益率 = r - 1

cm = (np.power(r, 250/len(data)) - 1) / abs(max_dd)
print(cm)

# method2: 通过收益率计算
ret = data[1:]/data[:-1] - 1  # ret.size == 99, data.size == 100
cm = (np.power(np.cumprod(ret+1)[-1], 250/len(data)) - 1) / abs(max_dd)
print(cm)

-0.10231682630000935
-0.10231682630000317


## Beta 和 Alpha

In [24]:
bmk = np.random.normal(size=100)
port = np.random.normal(size=100)

In [25]:
# 方法一：直接公式计算

# covariance
cov = np.cov(bmk, port)

beta = cov[0][1] / cov[0][0]
alpha = port.mean() - beta * bmk.mean()

print(cov)
print(beta, alpha)

[[ 1.10567626 -0.06244243]
 [-0.06244243  1.01178906]]
-0.056474426404903764 0.08815429469556636


In [38]:
# 方法二：调包
from sklearn import linear_model

# 建立线性回归模型
regr = linear_model.LinearRegression()
# 拟合
regr.fit(bmk.reshape(-1,1), port.reshape(-1,1))
# 不难得到直线的斜率、截距
beta, alpha = regr.coef_[0][0], regr.intercept_[0]

print(beta, alpha)

-0.05647442640490378 0.08815429469556636


## IC值

- 用 Numpy 方法排序：
在计算因子排名时，我们需要行因子值按照大小（从大到小）进行排序，并将排序后的结果转换为排名。在numpy库中，我们可以使用argsort函数来实现排序和排名的转换。

具体来说，argsort函数会返回原数组中每个元素在排序后的数组中的下标，从小到大排列。例如，对于数组[3,1,4,2]，argsort函数返回的结果为[1,3,0,2]，表示原数组中第二个元素（下标为1）在排序后的数组中排在第一位，第四个元素（下标为3）在排序后的数组中排在第二位，以此类推。

在计算因子排名时，我们需要将因子值从大到小排序，因此需要对原数组进行取反操作（即，-pe)，然后使用argsort函数进行排序。由于argsort函数返回的是下标，因此我们需要再次使用argsort将下标转换为排名。具体来说，我们可以使用argsort对排序后的下标进行排序，然后将排序后的结果加1，即可得到排名。

In [3]:
st.spearmanr(pd.Series(pe), pd.Series(returns))

SpearmanrResult(correlation=-0.02900857700857701, pvalue=0.3594690521683749)

In [2]:
# 声称股票收益率序列
np.random.seed(123)
returns = np.random.normal(0, 0.01, 1000)

# 生成PE序列
pe = np.random.normal(20, 5, 1000)

# 使用 Pandas 实现计算 ===============================
# 计算因子排名和股票收益率排名
pe_rank = pd.Series(pe).rank()
returns_rank = pd.Series(returns).rank()

# 计算IC值
ic = pe_rank.corr(returns_rank)
print("IC值：", ic)

# 使用 Numpy 实现计算 ===============================
# 计算因子排名和股票收益率排名
pe_rank = np.argsort(np.argsort(-pe))+1
returns_rank = np.argsort(np.argsort(-returns))+1

# 计算IC值
ic = np.corrcoef(pe_rank, returns_rank)[0,1]
print("IC值：", ic)

pd.DataFrame(columns=['returns','returns_rank','pe','pe_rank'], data=np.vstack((returns, returns_rank, pe, pe_rank)).T)

IC值： -0.02900857700857701
IC值： -0.02900857700857701


,returns,returns_rank,pe,pe_rank
0,-0.010856,842.0,16.255863,786.0
1,0.009973,158.0,22.837974,283.0
2,0.002830,366.0,23.590753,230.0
3,-0.015063,928.0,15.003096,857.0
4,-0.005786,713.0,22.374492,324.0
...,...,...,...,...
995,0.006348,257.0,24.228506,190.0
996,0.010699,134.0,14.400387,885.0
997,-0.009093,808.0,18.203516,653.0
998,0.004703,300.0,11.951525,948.0


In [18]:
from empyrical import sortino_ratio, max_drawdown, calmar_ratio

In [15]:
ret = np.diff(data) / data[:-1]
ret

array([-0.66688487,  0.16801453,  0.75970003,  1.24782735,  0.15397871,
       -0.51069744, -0.46916046,  2.35235792, -0.75570282,  0.37956859,
        1.12869714,  0.04829969, -0.34286981,  0.73947119, -0.13929717,
       -0.14064243, -0.22004157,  0.07516163, -0.69068954,  1.14224889,
       -0.26933981,  0.29340958,  0.89560962, -0.06871506, -0.27025755,
        1.71100805, -0.7904197 , -0.45851938,  1.36609314,  0.3851449 ,
        0.30635225, -0.38036178, -0.34668639,  0.6419897 ,  0.01995431,
       -0.23052271,  0.66406177,  0.15133239,  0.60140704, -0.53523371,
        1.21043728, -0.29892499, -0.45156306, -0.19703162,  1.25269799,
       -0.09919852, -0.69299273,  1.3730306 , -0.50995102,  0.70975091,
       -0.09384399,  0.02710357,  0.24708403, -0.43442926,  1.80048101,
       -0.63533675,  1.15597336,  1.17413493, -0.71002037,  2.05671142,
       -0.66477464,  1.75326087, -0.11792317,  0.04602543, -0.65316136,
        0.86301639, -0.38241303,  1.00825009, -0.21754608,  0.02

In [17]:
max_drawdown(ret)

-0.8865163316678304

In [20]:
calmar_ratio(ret, period='daily', annualization=None)

-0.335530194652632

In [22]:
def calculate_annualized_calmar_ratio(daily_returns):
    cumulative_return = (1 + sum(daily_returns)) ** (252 / len(daily_returns)) - 1  # 假设一年有252个交易日
    max_drawdown = max_drawdown_calculation(daily_returns)
    print(max_drawdown)
    calmar_ratio = cumulative_return / abs(max_drawdown)
    annualized_calmar_ratio = calmar_ratio * (12 ** 0.5)  # 年化调整因子，假设每年有12个月
    return annualized_calmar_ratio

def max_drawdown_calculation(daily_returns):
    max_drawdown = 0
    peak = 0
    for daily_return in daily_returns:
        peak = max(peak, peak + daily_return)
        drawdown = peak - (peak + daily_return)
        max_drawdown = min(max_drawdown, drawdown)
    return max_drawdown

# 示例数据，每日收益率列表
ret

# 计算年化Calmar比率
annualized_calmar_ratio = calculate_annualized_calmar_ratio(ret)

print("年化Calmar比率：", annualized_calmar_ratio)

-2.7813959911904576
年化Calmar比率： 3477.6726189447218
